# 🎤 Text to Speech(TTS) 실습 강의 노트

- [OpenAI.fm](https://openai.fm) 데모

## 환경 준비 및 API 키 설정

In [ ]:
pip install openai

- Colab 메뉴 `보안 비밀(열쇠 아이콘)`에 `OPENAI_API_KEY`를 저장해 두었다면 자동으로 불러옵니다.
- 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.

In [ ]:
import os

OPENAI_API_KEY = None
try:
    from google.colab import userdata  # type: ignore
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
except Exception:
    print("Colab 메뉴의 보안 비밀(열쇠 아이콘)에서 환경 변수가 제대로 저장되었는지 확인해주세요.")

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
print("✅ API 키 설정 완료")

In [ ]:
from openai import OpenAI, AsyncOpenAI
from pathlib import Path

client = OpenAI()
async_client = AsyncOpenAI()
print("✅ 클라이언트 생성 완료")


## 2) Text To Speech (TTS)

TTS는 텍스트를 오디오로 변환하는 모델입니다.

OpenAI API에는 11개의 내장 음성이 포함되어 있으며 다음과 같이 사용할 수 있습니다:
- 작성된 블로그 포스트를 나레이션하기
- 다국어로 음성 오디오 생성하기
- 스트리밍을 사용한 실시간 오디오 출력 제공하기

## 주요 파라미터

**model**: 사용 가능한 TTS 모델 중 하나를 지정합니다. `tts-1`, `tts-1-hd`, `gpt-4o-mini-tts` 중 선택 가능합니다.
- 실습 모델: gpt-4o-mini-tts

**input**: 오디오를 생성할 텍스트입니다. 최대 길이는 4096자입니다.

**voice**: 오디오를 생성할 때 사용할 음성입니다. 지원되는 음성은 `alloy`, `ash`, `ballad`, `coral`, `echo`, `fable`, `onyx`, `nova`, `sage`, `shimmer`, `verse` 입니다. 음성의 미리듣기는 [Text to speech guide](https://platform.openai.com/docs/guides/text-to-speech)에서 확인할 수 있습니다.

**instructions**: 생성된 오디오의 음성을 추가 지시사항으로 제어합니다. 억양, 감정, 속도 등을 지정할 수 있습니다. `tts-1` 또는 `tts-1-hd` 모델에서는 작동하지 않습니다.

**response_format**: 오디오 출력 형식을 지정합니다. 지원되는 형식은 `mp3`(기본값), `opus`, `aac`, `flac`, `wav`, `pcm` 입니다.

**speed**: 생성된 오디오의 속도입니다. 0.25에서 4.0 사이의 값을 선택할 수 있습니다. 기본값은 1.0입니다.

**stream_format**: 오디오 스트리밍 형식을 지정합니다. 지원되는 형식은 `audio`(기본값), `sse` 입니다. `sse`는 `tts-1` 또는 `tts-1-hd` 모델에서는 지원되지 않습니다.

In [ ]:
from pathlib import Path
from openai import OpenAI

client = OpenAI()
speech_file_path = Path(__file__).parent / "speech.mp3" # 경로는 좀 체크 해봐야함

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="와! 오늘 같은 날은 LLM 실습 배우기에 완전히 최고의 날이에요! 정말 신나요!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

## 3) 기본 텍스트→음성 변환 데모 (파일 저장)
가장 기본적인 워크플로우는 텍스트와 음성 설정을 넘겨 MP3 파일로 저장하는 것입니다.


In [ ]:
speech_file = Path("speech.mp3")
speech_file.unlink(missing_ok=True)

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="coral",
    input="Today is a wonderful day to build something people love!",
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file)

print("🎧 음성 파일 저장 완료 ->", speech_file)


> **설명 포인트**
> - `with_streaming_response.create` 컨텍스트를 사용하면 서버로부터 오는 오디오 스트림을 실시간으로 파일에 쓸 수 있습니다.
> - 음성 합성 요청마다 `voice`, `instructions`를 통해 억양/감정/속도 등을 제어할 수 있습니다.



## 4) 음성 옵션(voice) · 프롬프트 제어 · 다국어 지원
- **지원 음성(11종)**: `alloy`, `ash`, `ballad`, `coral`, `echo`, `fable`, `nova`, `onyx`, `sage`, `shimmer` 등 (최신 목록은 OpenAI.fm에서 확인)
- **프롬프트 제어**: 인사톤, 속도, 감정, 억양, 속삭임 등을 `instructions`에 명시
- **언어 지원**: Whisper 모델 수준의 다국어 텍스트 입력을 지원 (음성은 영어에 최적화되어 있으나 대부분의 언어 입력 가능)
- **정책 안내**: 음성이 AI 생성임을 사용자에게 반드시 고지해야 합니다.


## 5) 스트리밍 재생 및 실시간 활용
TTS 응답을 바로 재생하려면 스트리밍 응답과 로컬 플레이어를 결합할 수 있습니다.


In [ ]:
import asyncio
from openai.helpers import LocalAudioPlayer

async def play_streaming_tts(text: str):
    async with async_client.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input=text,
        instructions="Speak in a cheerful and positive tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

# asyncio.run(play_streaming_tts("Today is a wonderful day to build something people love!"))


> **설명 포인트**
> - `response_format="wav"` 또는 `"pcm"`을 사용하면 지연을 줄이고 빠르게 스트리밍 재생할 수 있습니다.
> - 웹/모바일 앱에서는 스트리밍 응답을 스피커로 직접 송출하거나 음성 채팅에 활용할 수 있습니다.



## 6) 출력 포맷 비교와 응용 예시
| 포맷 | 용도 | 특징 |
| --- | --- | --- |
| `mp3` (기본) | 일반적인 배포/저장 | 범용성 높음, 용량 작음 |
| `opus` | 인터넷 스트리밍 | 낮은 지연, 적은 비트레이트 |
| `aac` | 모바일/미디어 플랫폼 | YouTube·iOS·Android 선호 |
| `flac` | 보존·아카이빙 | 무손실 압축, 용량 큼 |
| `wav` | 저지연 재생 | 헤더 있는 비압축, 디코딩 필요 없음 |
| `pcm` | 실시간 파이프라인 | 헤더 없는 원시 샘플(24kHz/16bit) |

> **응용 아이디어**: 프롬프트 자동화로 블로그 TTS, 다국어 오디오 가이드, 실시간 음성 피드백 시스템 등을 구현할 수 있습니다.


> 💡 **강의 마무리**
> - Week2 실습 노트(`03-TTS-Practice.ipynb` 예정)에서 TODO 형태로 동일한 단계(파일 저장, 스트리밍)를 구현하도록 안내하세요.
> - 음성 생성은 빠르므로 라이브 데모가 가능하지만, 다양한 음성/언어 결과를 비교하는 샘플을 미리 준비하면 Discussion에 도움이 됩니다.

레퍼런스
[Audio API 문서](https://platform.openai.com/docs/api-reference/audio/createSpeech)
[OpenAI.fm](https://openai.fm) 데모